# Overview
The purpose of the code below is to automatically generate terms in the Lagrangian of N scalar fields with O(N) and Lorentz symmetry. It does so by defining a "field" class, instances of which represent everything from individual field operators, field derivatives, and arbitrary contractions of these basic components, which make up terms in the Lagrangian. A field object is required to possess at minimum the structure needed to determine the algebraic composition properties of fields in the generation of terms of a given dimension in the Lagrangian - namely, a list of the elementary fields $\phi^{i}$ and their derivatives $\partial_{\mu} \phi^{i}$ that appear in the term, and the contractions among the elementary fields making up the term. 

## General Strategy

The overall strategy of the above code is first to generate "fully connected" terms (i.e., terms that cannot be factorized as a simple product of scalar terms) up to a given mass dimension $D$, and then to generate all terms up to mass dimension $D$ by combining the fully connected terms in all possible ways that lead to terms with mass dimension less than or equal to $D$. 



## Explanation of main class attributes

The basic attributes of class instances are

1) **elementary_str**: An ordered list of strings representing the elementary fields appearing in the term, and the structure of their contractions.

2) **elementary**: A list of field objects corresponding to those in *elementary_str*. This is generated automatically by the *set_elementary* method, given a specification of elements in *elementary_str*.

3) ** contractions**: A list of the contractions among adjacent field objects, indicating whether they are contracted along the Lorentz index, the O(N) index, neither, or both. A contraction in the Lorentz index is represented by $[1,0]$, a contraction in the O(N) index by $[0,1]$, a contraction in both by $[1,1]$, and a contraction in neither by $[0,0]$.

4) **connected**: A list of the fully connected fields (i.e., those that cannot be factored) that make up the term. This is useful for the method *generate*, which produces terms that are factorizable in terms of fully connected terms. 

## Explanation of main class methods

The most important class methods are

1) **print_Lagrangian**: takes as input the mass dimension up to which one wishes to print terms of the given Lagrangian. This relies on the *generate* class method. 

2) **generate**: this generates a list of field objects, one for each term in the Lagrangian, up to a given mass dimension D. This relies on the *generate_connected* class method. 

3) **generate_connected**: this generates a list of "connected" terms up to a specified mass dimension D - i.e. terms with no un-contracted indices. 

## Notation
NOTE: Summation over repeated indices assumed. Fully connected terms indicated in parentheses

1) $P \equiv \phi^{i}$

2) $DP \equiv \partial_{\mu} \phi^{i}$

3) $(PP) \equiv \phi^{i} \phi^{i}$ 

4) $(DPDP) \equiv \partial_{\mu} \phi^{i} \partial_{\mu} \phi^{i}$

5) $PDP \equiv \phi^{i}\partial_{\mu} \phi^{i}$

6) $(PDPDPP) \equiv \phi^{i_{1}}\ \partial_{\mu_{1}} \phi^{i_{1}} \ \partial_{\mu_{1}} \phi^{i_{2}} \ \phi^{i_{2}} $

7) $(PDPDPDPDPP) \equiv \phi^{i_{1}}\ \partial_{\mu_{1}} \phi^{i_{1}} \ \partial_{\mu_{1}} \phi^{i_{2}}\ \partial_{\mu_{2}} \phi^{i_{2}} \ \partial_{\mu_{2}} \phi^{i_{3}} \ \phi^{i_{3}} $

8) etc. 

In [137]:
from math import floor

class field(object):
    def __init__(self, elementary_str, contractions, symbol):
        self.elementary_str = elementary_str #Stores list of N symbols for elementary field factors phi and dphi
        self.elementary = []
        self.contractions = contractions #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.symbol = symbol
        #self.generated_terms = []
        self.connected = [] #list of connected terms, includes repeats
    def __str__(self):
        return 'elementary_str: ' + str(self.elementary_str) + '\n' \
            + 'contractions: ' + str(self.contractions) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'massDim: ' + str(self.massDim()) + '\n' \
            + 'LORrank: ' + str(self.LORrank()) + '\n' \
            + 'ONrank: ' + str(self.ONrank()) + '\n' \
            + 'symbol: ' + str(self.make_symbol())
    def __eq__(self, other):
        eq = (self.elementary_str == other.elementary_str) & (self.contractions == other.contractions)
        return eq
    
    #Define get and set functions
    def get_elementary_str(self):
        return self.elementary_str
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_symbol(self):
        return self.symbol
    def get_connected(self):
        return self.connected
    
    def set_elementary_str(self, elementary_str):
        self.elementary_str = elementary_str
    def set_elementary(self): #should run this to set list of constituent fields
        for symbol in self.elementary_str:
            if symbol == 'P':
                field1 = field(['P'],[], 'P')
                self.elementary.append(field1)
            elif symbol == 'DP':
                field1_deriv = field(['DP'],[], 'DP')
                self.elementary.append(field1_deriv)
            else:
                print('Error: Symbol must be P or DP.')
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_symbol(self, symbol):
        self.symbol = symbol
    def set_connected(self, connected_list):
        self.connected = connected_list
    
    
    def make_symbol(self): #use for connected terms
        symbol_str = '('
        for i in range(len(self.get_elementary_str())):
            symbol_str = symbol_str + self.get_elementary_str()[i]
            if i < len(self.get_contractions()) and self.contractions[i] == [0,0]:
                symbol_str = symbol_str + ')('
        symbol_str = symbol_str + ')'
        
        return symbol_str
            
    
    def print_connected(self):
        for term in self.get_connected():
            print(term)
    
    def massDim(self):
        
        counter = 0
        
        for field in self.get_elementary():
            if field.get_symbol() == 'P':
                counter = counter + 1
            elif field.get_symbol() == 'DP':
                counter = counter + 2
        
        return counter                       

    def LORrank(self):
        '''
        To get LORrank Count number of Lorentz contractions (counter) in contractions. Multiply by 2. Subtract this from 
        the number of Lorentz indices appearing in elementary - namely, the number of 'DP' fields. 
        '''
        counter1 = 0
        for pair in self.get_contractions():
            if pair[0] == 1:
                counter1 = counter1 + 1
            else:
                pass
            
        counter2 = 0
        for field in self.get_elementary():
            if field.get_symbol() == 'DP':
                counter2 = counter2 + 1
            else:
                pass     
        
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def ONrank(self):
        
        counter1 = 0
        
        #Number of contractions is the number of time a '1' appears in the second index of pairs in contractions
        for pair in self.get_contractions():
            if pair[1] == 1:
                counter1 = counter1 + 1
            else:
                pass
        
        #Number of ON indices is just the number of fields in self.elementary
        counter2 = len(self.get_elementary_str())
            
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def contract(self, other, contraction, symbol_prod):
        '''
        other: elementary field, P or DP
        contraction: pair of integers - [1,0] for contraction in LOR, [0,1] for contraction in ON, [1,1] for both
        symbol_prod: string
        '''
        
        #print("contraction: " + str(contraction))
        
        if contraction == [1,0]:
            contractions_prod = self.get_contractions() + [[1,0]]
        elif contraction == [0,1]:
            contractions_prod = self.get_contractions() + [[0,1]]
        elif contraction == [1,1]:
            contractions_prod = self.get_contractions() + [[1,1]]
        else:
            print("Error: contractions must be [1,0], [0,1], or [1,1]")
            
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    def bubbleSort(self, field_list): #sorts list of connected fields in ascending order by massDim. 
        n = len(field_list)
 
        # Traverse through all array elements
        for i in range(n):
 
            # Last i elements are already in place
            for j in range(0, n-i-1):
 
                # traverse the array from 0 to n-i-1
                # Swap if the element found is greater
                # than the next element
                if field_list[j].massDim() > field_list[j+1].massDim():
                    field_list[j], field_list[j+1] = field_list[j+1], field_list[j]
    
        return field_list
    
    
    def multiply(self, other, symbol_prod=''):
        '''
        Multiplies without contraction
        INPUT:
        self: field object
        other: field object
        symbol_prod: symbol for new field obj output
        
        OUTPUT:
        prod: field object, concatenating self and other, by inserting [0,0] in contractions
        
        '''
        if (not self.get_elementary_str()) or (not other.get_elementary_str()):
            #print("one factor is const")
            contractions_prod = self.get_contractions() + other.get_contractions()
        else:
            contractions_prod = self.get_contractions() + [[0,0]] + other.get_contractions()
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        connected_prod = self.get_connected() + other.get_connected()
        #Sort connected_prod by mass dimension, with lowest first and highest last
        connected_prod_sorted = field.bubbleSort(self, connected_prod)
        
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        prod.set_connected(connected_prod_sorted)
        
        return prod
    
    
    
    def generate_connected(self, d):
        '''
        INPUT
        d: non-negative integer, mass dimension up to which to generate connected field terms
        
        OUTPUT
        terms: list of field terms of mass dimension no larger than d.  
        '''
        
        const = field([],[],'1')
        const.set_connected([field([],[],'1')])
        terms = [const]
        
        
        if d >= 2:
            #print("in d>=2")
            field1 = field(['P'],[],'P')
            field2 = field(['P'],[],'P')
            prod = field.contract(field1, field2, [0,1], 'PP')
            prod.set_connected([field.contract(field1, field2, [0,1], 'PP')])
            terms.append(prod)
        if d >= 4:
            deriv1 = field(['DP'],[],'DP')
            deriv2 = field(['DP'],[],'DP')
            prod = field.contract(deriv1, deriv2, [1,1], 'DPDP')
            prod.set_connected([field.contract(deriv1, deriv2, [1,1], 'DPDP')])
            terms.append(prod)
        
        field1 = field(['P'],[],'P')
        deriv1 = field(['DP'],[],'DP')
        
        if d < 4:
            return terms
    
        
        root = field.contract(field1, deriv1, [0,1], 'PDP')
        
        while root.massDim() <= d:
            
            if root.LORrank() != 0:
                deriv1 = field(['DP'],[],'DP')
                root = field.contract(root, deriv1, [1,0], '')
                
            
            if root.ONrank() != 0:
                field1 = field(['P'],[],'P')
                deriv1 = field(['DP'],[],'DP')
                new = field.contract(root, field1, [0,1], '')
                if new.massDim() <= d and new.LORrank() == 0 and new.ONrank() == 0:
                    new.set_connected([field.contract(root, field1, [0,1], '')])
                    terms.append(new)
                
                root = field.contract(root, deriv1, [0,1], '')   
        
        return terms
    
   
    
    def generate(self, D):
        if D < 2:
            print("D should be an integer >= 2.")
            return []
        else:
            #initiate list of generated terms
            generated = []
            
            #build list of connected terms
            connected_terms_D = self.generate_connected(D)
            
            #const = field([],[],'')
            #const.set_connected([field([],[],'')])
            
            #initiate cache
            cache = []
            for term in connected_terms_D:
                cache.append(term)
            
            for i in range(floor(D/2)-1):
                
                #for term in cache:
                    #print(term.get_elementary_str())
                
                cache_len = len(cache)
                
                for j in range(len(cache)):
                    for k in range(len(connected_terms_D)):
                        if connected_terms_D[k].massDim() <= cache[j].get_connected()[0].massDim():
                            prod = field.multiply(cache[j], connected_terms_D[k])
                            cache.append(prod)
                        else: 
                            pass
                cache = cache[cache_len:]
                
            #Filter out terms in cache that have massDim <= D   
            for term in cache:
                if term.massDim() <= D:
                    generated.append(term)
                else:
                    pass
                
            generated = self.bubbleSort(generated)    
            return generated
        
    def print_Lagrangian(self, D):
            
            L = ''
            
            terms = self.generate(D)
            
            for i in range(len(terms)):
                L = L + ' c_'+ str(i) + terms[i].make_symbol() + ' +' 
            
            print("L = " + L)

In [138]:
A = field([],[],'') #Declare field object for use of class methods. 

In [139]:
A.set_elementary() #Create list of elementary fields (here, the constant field).

In [140]:
generated_list = A.generate(10)

In [141]:
print(len(generated_list))

17


In [142]:
for term in generated_list:
    print(term)
    print('')

elementary_str: []
contractions: []
symbol: 
massDim: 0
LORrank: 0
ONrank: 0
symbol: ()

elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: 
massDim: 2
LORrank: 0
ONrank: 0
symbol: (PP)

elementary_str: ['P', 'P', 'P', 'P']
contractions: [[0, 1], [0, 0], [0, 1]]
symbol: 
massDim: 4
LORrank: 0
ONrank: 0
symbol: (PP)(PP)

elementary_str: ['DP', 'DP']
contractions: [[1, 1]]
symbol: 
massDim: 4
LORrank: 0
ONrank: 0
symbol: (DPDP)

elementary_str: ['P', 'P', 'P', 'P', 'P', 'P']
contractions: [[0, 1], [0, 0], [0, 1], [0, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
symbol: (PP)(PP)(PP)

elementary_str: ['DP', 'DP', 'P', 'P']
contractions: [[1, 1], [0, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
symbol: (DPDP)(PP)

elementary_str: ['P', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 6
LORrank: 0
ONrank: 0
symbol: (PDPDPP)

elementary_str: ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P']
contractions: [[0, 1], [0, 0], [0, 1], [0, 0], [0, 1], [0, 0], [

In [143]:
A = field([],[],"")

In [144]:
A.print_Lagrangian(0)

D should be an integer >= 2.
L = 


In [152]:
A.print_Lagrangian(6)

L =  c_0() + c_1(PP) + c_2(PP)(PP) + c_3(DPDP) + c_4(PP)(PP)(PP) + c_5(DPDP)(PP) + c_6(PDPDPP) +


In [146]:
A.print_Lagrangian(6)

L =  c_0() + c_1(PP) + c_2(PP)(PP) + c_3(DPDP) + c_4(PP)(PP)(PP) + c_5(DPDP)(PP) + c_6(PDPDPP) +


In [147]:
A.print_Lagrangian(12)

L =  c_0() + c_1(PP) + c_2(PP)(PP) + c_3(DPDP) + c_4(PP)(PP)(PP) + c_5(DPDP)(PP) + c_6(PDPDPP) + c_7(PP)(PP)(PP)(PP) + c_8(DPDP)(PP)(PP) + c_9(DPDP)(DPDP) + c_10(PDPDPP)(PP) + c_11(PP)(PP)(PP)(PP)(PP) + c_12(DPDP)(PP)(PP)(PP) + c_13(DPDP)(DPDP)(PP) + c_14(PDPDPP)(PP)(PP) + c_15(PDPDPP)(DPDP) + c_16(PDPDPDPDPP) + c_17(PP)(PP)(PP)(PP)(PP)(PP) + c_18(DPDP)(PP)(PP)(PP)(PP) + c_19(DPDP)(DPDP)(PP)(PP) + c_20(DPDP)(DPDP)(DPDP) + c_21(PDPDPP)(PP)(PP)(PP) + c_22(PDPDPP)(DPDP)(PP) + c_23(PDPDPP)(PDPDPP) + c_24(PDPDPDPDPP)(PP) +


# TO DO:  
1) Involve fermions (including gamma matrix algebra), and anticommuting fields. Maybe do a scalar-fermi theory.
 
2) Then go to gauge theories: QED, then Eletroweak, then QCD, then full SM. 

3) So far, symmetry is inserted only by virtue of requiring O(N) and Lorentz indicies to be fully contracted in all terms (from an algebraic point of view, this is the implication of the symmetry for the form of allowable terms in the Lagrangian. Perhaps there is a way to somehow insert the symmetry in a more fundamental way (e.g., via group generators for continuous symmetries and via other methods for discrete symmetries) and then derive the invariants somehow. 
 